# 0.0. Imports

In [1]:
import requests
import pandas as pd
import numpy  as np
import pickle
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model    import ElasticNet
from sklearn.metrics         import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing   import MinMaxScaler

## 0.1. Loading Data

In [2]:
csv_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
df_raw = pd.read_csv( csv_url, sep=';' )

# 1.0. Data Preparation

In [3]:
df1 = df_raw.copy()

## 1.1. Rescaling

In [4]:
# min max scaler
# x_new = ( x_old - min() ) / ( max - min )
mms = MinMaxScaler()

In [5]:
# free sulfur dioxide
df1['free sulfur dioxide'] = mms.fit_transform( df1[['free sulfur dioxide']].values )
pickle.dump( mms, open( '/Users/meigarom/repos/SejaUmDataScientist/deploy/free_sulfur_scaler.pkl', 'wb' ))

# total sulfur dioxide
df1['total sulfur dioxide'] = np.log1p( df1['total sulfur dioxide'] )
df1['total sulfur dioxide'] = mms.fit_transform( df1[['total sulfur dioxide']].values )
pickle.dump( mms, open( '/Users/meigarom/repos/SejaUmDataScientist/deploy/total_sulfur_scaler.pkl', 'wb' ))

In [6]:
train, test = train_test_split( df1 ) #75% e 25% test

In [7]:
# train and test dataset
x_train = train.drop( 'quality', axis=1 )
y_train = train['quality']

x_test = test.drop( 'quality', axis=1 )
y_test = test['quality']

# 2.0. ML Modelling

In [8]:
# model definition
model = ElasticNet( alpha=0.5, l1_ratio=0.5, random_state=42 )

In [9]:
# training
model.fit( x_train, y_train )

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=42, selection='cyclic', tol=0.0001, warm_start=False)

In [10]:
# prediction
pred = model.predict( x_test )

In [11]:
rmse = np.sqrt( mean_squared_error( y_test, pred ))
mae = mean_absolute_error( y_test, pred )
r2  = r2_score( y_test, pred )

In [12]:
print( 'RMSE: {}'.format( rmse ) )
print( 'MAE: {}'.format( mae ) )
print( 'R2: {}'.format( r2 ))

RMSE: 0.7624371623577465
MAE: 0.6315322325060694
R2: 0.11445508233701251


In [13]:
# save trained model
pickle.dump( model, open( '/Users/meigarom/repos/SejaUmDataScientist/deploy/model_wine_quality.pkl', 'wb' ) )

In [14]:
!ls deploy

__pycache__             handler.py              total_sulfur_scaler.pkl
free_sulfur_scaler.pkl  model_wine_quality.pkl  wine_quality.py


# 5.0. Prediction 

In [15]:
df = df_raw.drop( 'quality', axis=1 ).sample()

In [16]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
217,8.1,0.725,0.22,2.2,0.072,11.0,41.0,0.9967,3.36,0.55,9.1


In [26]:
df_json = df.to_json( orient='records' )

In [27]:
url = 'http://0.0.0.0:5000/predict'
data = df_json
header = {'Content-type': 'application/json'}

# Request
r = requests.post( url=url, data=data, headers=header )

In [28]:
pd.DataFrame( r.json(), columns=r.json()[0].keys() )

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,prediction
0,8.1,0.725,0.22,2.2,0.072,0.140845,10.487217,0.9967,3.36,0.55,9.1,5.495551


## Class In Production

In [17]:
import pickle 

class WineQuality( object ):
    def __init__( self ):
        self.free_sulfur_scaler = pickle.load( open( '/Users/meigarom/repos/SejaUmDataScientist/deploy/free_sulfur_scaler.pkl', 'rb'))
        self.total_sulfur_scaler = pickle.load( open( '/Users/meigarom/repos/SejaUmDataScientist/deploy/total_sulfur_scaler.pkl', 'rb'))
        
    
    def data_preparation( self, df ):
        # rescaling free sulfur
        df['free sulfur dioxide'] = self.free_sulfur_scaler.transform( df['free sulfur dioxide'] )
        
        # rescaling total sulfur
        df['total sulfur dioxide'] = self.total_sulfur_scaler.transform( df['total sulfur dioxide'] )
        
        return df